In [1]:
%matplotlib inline

import gym
import itertools
import matplotlib
import numpy as np
import sys
import tensorflow as tf
import collections


from envs.cliff_walking import CliffWalkingEnv
import plotting

matplotlib.style.use('ggplot')

In [6]:
env = CliffWalkingEnv()
env.__dict__

{'P': {0: {0: [(1.0, 0, -1.0, False)],
   1: [(1.0, 1, -1.0, False)],
   2: [(1.0, 12, -1.0, False)],
   3: [(1.0, 0, -1.0, False)]},
  1: {0: [(1.0, 1, -1.0, False)],
   1: [(1.0, 2, -1.0, False)],
   2: [(1.0, 13, -1.0, False)],
   3: [(1.0, 0, -1.0, False)]},
  2: {0: [(1.0, 2, -1.0, False)],
   1: [(1.0, 3, -1.0, False)],
   2: [(1.0, 14, -1.0, False)],
   3: [(1.0, 1, -1.0, False)]},
  3: {0: [(1.0, 3, -1.0, False)],
   1: [(1.0, 4, -1.0, False)],
   2: [(1.0, 15, -1.0, False)],
   3: [(1.0, 2, -1.0, False)]},
  4: {0: [(1.0, 4, -1.0, False)],
   1: [(1.0, 5, -1.0, False)],
   2: [(1.0, 16, -1.0, False)],
   3: [(1.0, 3, -1.0, False)]},
  5: {0: [(1.0, 5, -1.0, False)],
   1: [(1.0, 6, -1.0, False)],
   2: [(1.0, 17, -1.0, False)],
   3: [(1.0, 4, -1.0, False)]},
  6: {0: [(1.0, 6, -1.0, False)],
   1: [(1.0, 7, -1.0, False)],
   2: [(1.0, 18, -1.0, False)],
   3: [(1.0, 5, -1.0, False)]},
  7: {0: [(1.0, 7, -1.0, False)],
   1: [(1.0, 8, -1.0, False)],
   2: [(1.0, 19, -1.0, Fals

In [3]:
class PolicyEstimator():
    """
    Policy Function approximator.
    
    A policy estimator is a function that produce the following distribution
    π(a|s, θ)
    
    s is the current state
    a is an action corresponding 
    θ is the set of parameters, in this case, it is a neural network
    
    Each time, we have two inputs:
        - A state generated by Monte Carlo
        - An action generated by Monte Carlo
    The model would produce (from the state) an output layer
    that correspond to a preference function h(s,a).
    
    Following is the formula for updating:
    
    θ += α*γ^t*G * DELTA_θ (log π(At|St, θ))
    
    or, more succintly, for baseline also:
    
    θ += α * TARGET_WEIGHT * DELTA_θ (log π(At|St, θ))
    
    where TARGET_WEIGHT could be the (discounted) return from time t
    adjusted with baseline.
    
    This corresponding to the following graph mechanism:
    
    The graph basically predict the policy action
    """
    
    def __init__(self, learning_rate=0.01, scope="policy_estimator"):
        """
        The code to declare your tensorflow graph comes here
        """
        with tf.variable_scope(scope): 
            "Declare all placeholders"
            "Placeholder for input"
            """
            Now in this case, if the state is represented as a number ranking from
            start point of the map to the end, we lost the locality between
            cells of two consecutive rows.
            So let's make it a row and column
            """
            # No batch
            self.state = tf.placeholder([2], name="state", dtype = tf.int32)
            
            "Placeholder for Monte Carlo action"
            self.action = tf.placeholder(name="action", dtype = tf.int32)
            
            "Placeholder for target"
            self.target = tf.placeholder(name="target", dtype = tf.int32)
            
            "Placeholder for action"
            
            "Declare all nodes in the graph"
    
    def predict(self, state, sess=None):
        """
        """
        pass
    
    def update(self, state, target, action, sess=None):
        """
        state: input state
        target: final return of an episode
        action: input action
        """
        pass

In [4]:
class ValueEstimator():
    """
    Value Function approximator. 
    """
    
    def __init__(self, learning_rate=0.1, scope="value_estimator"):
        """
        """       
    
    def predict(self, state, sess=None):
        """
        """

    def update(self, state, target, sess=None):
        """
        """

In [5]:
def reinforce(env, estimator_policy, estimator_value, num_episodes, discount_factor=1.0):
    """
    REINFORCE (Monte Carlo Policy Gradient) Algorithm. Optimizes the policy
    function approximator using policy gradient.
    
    Args:
        env: OpenAI environment.
        estimator_policy: Policy Function to be optimized 
        estimator_value: Value function approximator, used as a baseline
        num_episodes: Number of episodes to run for
        discount_factor: Time-discount factor
    
    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """
    pass

In [8]:
tf.reset_default_graph()

global_step = tf.Variable(0, name="global_step", trainable=False)
policy_estimator = PolicyEstimator()
value_estimator = ValueEstimator()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Note, due to randomness in the policy the number of episodes you need to learn a good
    # policy may vary. ~2000-5000 seemed to work well for me.
    stats = reinforce(env, policy_estimator, value_estimator, 2000, discount_factor=1.0)